In [2]:
from ultralytics import YOLO
import cv2
import numpy as np

In [3]:
model = YOLO("yolov8n.pt")

In [58]:
cap = cv2.VideoCapture("train.mp4")

ret, frame = cap.read()
height, width = frame.shape[:2]
line_x = width // 2
line_y = height // 2

id_positions = {}           
counted_up = set()
counted_down = set()
count_up = 0
count_down = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, classes=[0], tracker="botsort.yaml", verbose=False)

    annotated_frame = frame.copy()

    # cv2.line(annotated_frame, (line_x + 200, 0), (line_x + 200, height), (0, 0, 255), 2)
    cv2.line(annotated_frame, (0, line_y + 150), (width, line_y + 150), (0, 0, 255), 2)

    if results[0].boxes.id is not None:
        for box in results[0].boxes:
            track_id = int(box.id[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"ID {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            cv2.circle(annotated_frame, (center_x, center_y), 4, (255, 0, 0), -1)

            if track_id in id_positions:
                prev_y = id_positions[track_id]
                id_positions[track_id] = center_y

                if prev_y < line_y + 150 and center_y >= line_y + 150 and track_id not in counted_down:
                    count_down += 1
                    counted_down.add(track_id)

                if prev_y > line_y + 150 and center_y <= line_y + 150 and track_id not in counted_up:
                    count_up += 1
                    counted_up.add(track_id)

            else:
                id_positions[track_id] = center_y 

    cv2.putText(annotated_frame, f"up  ->  down: {count_down}", (200, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(annotated_frame, f"down <-  up: {count_up}", (200, 180),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("People Counter", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
